In [51]:
%load_ext autoreload
%autoreload 2
import ast
import openai
import re
import os
# openai.api_key = os.getenv('OPENAI_KEY')

from document import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.Completion.create(
  model="text-davinci-003",
  prompt="Say this is a test",
  max_tokens=7,
  temperature=0
)

In [28]:
import time
def generate_docstring(function, attempts = 0):
    """
    Generates a docstring for a given function signature and description
    using the OpenAI GPT-3 API.
    """
    prompt = f'Generate a concise but informative docstring for the following function:\n\n{function}\n\n'

    try:
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo', 
            messages = [
                {'role': 'system', 'content': 'You are an expert programmer with 20+ years of experience writing clean and efficient Python code.'},
                {'role': 'user', 'content': prompt}
            ], 
            max_tokens = 200)
    except openai.error.APIError as e:
        if attempts > 3:
            raise e
        generate_docstring(function, attempts+1)
    except openai.error.RateLimitError as e:
        if attempts > 3:
            raise e
        print('Hit rate limit. Waiting 20 seconds')
        time.sleep(21)
        generate_docstring(function, attempts+1)
    return response

In [69]:
add_docstrings_to_file('./download_nyt_recipes.py')

Hit rate limit. Waiting 20 seconds
Hit rate limit. Waiting 20 seconds


True

In [70]:
l = extract_functions('./cheffrey.py')

In [73]:
function= l[0]

In [74]:
docstring = generate_docstring(ast.unparse(function))

In [75]:
docstring

'"""\nLoad a YAML file from the \'data\' directory with the given name.\n\nParameters:\n    name (str): The name of the YAML file to load. Valid types are \'config\' and \'recipes\'.\n\nReturns:\n    The contents of the YAML file as a Python object.\n\nRaises:\n    ValueError: If the given name is not recognized as a valid type.\n\nNote:\n    This function requires the `yaml` module to be installed.\n"""'

In [76]:
function.body.insert(0, ast.parse(docstring).body[0])

In [78]:
ast.unparse(function)

'def load_yaml(name):\n    """\nLoad a YAML file from the \'data\' directory with the given name.\n\nParameters:\n    name (str): The name of the YAML file to load. Valid types are \'config\' and \'recipes\'.\n\nReturns:\n    The contents of the YAML file as a Python object.\n\nRaises:\n    ValueError: If the given name is not recognized as a valid type.\n\nNote:\n    This function requires the `yaml` module to be installed.\n"""\n    valid_names = [\'config\', \'recipes\']\n    if name in valid_names:\n        with open(ROOT_DIR / f\'data/{name}.yaml\', \'r\') as f:\n            return yaml.safe_load(f)\n    else:\n        raise ValueError(f\'Unrecognized name: {name}. Valid types are: {valid_names}\')'

In [81]:
file_path = './download_nyt_recipes.py'
def add_docstrings_to_file(file_path):
    """
    Adds docstrings to all functions in a file.
    """
    # Extract functions from file
    functions = extract_functions(file_path)

    # Add docstrings to each function
    for function in functions:
        print(function.name)
        function = add_docstring(function)

    # Rewrite file with modified functions
    with open(file_path, "r+") as file:
        source = file.read()
        module = ast.parse(source)
        ast.fix_missing_locations(module)
        for function in functions:
            for i, node in enumerate(module.body):
                if node is function:
                    module.body[i] = function
                    break
        file.seek(0)
        file.write(ast.unparse(module))
        file.truncate()
add_docstrings_to_file(file_path)

SyntaxError: invalid syntax (<unknown>, line 1)